This notebook will be used for a coursera captone: Battle of the Neighborhoods

# Installing/Importing Necessary Libraries and Credentials

In [1]:
#install necessary modules
!pip install bs4
!pip install requests
!pip install html5lib
!pip install geocoder
#import used libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import io

In [2]:
#Based on week2 lab, saving this infor in cell for use later

CLIENT_ID = 'UATKCT2MMK3S1ZP11HH3BR0YJ42W1ELEZMEVXD0PLWGGAPFO' # your Foursquare ID
CLIENT_SECRET = 'HTVNXNE1GPEFQ4RJKPX3BIAR1BYG3WYQQ3KZSFUFF2MLBRHN' # your Foursquare Secret
ACCESS_TOKEN = 'M4LRXZBVTKQ3KORATCGE5SLAU4UXMZM0DTMSBPYINLG5BLVD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 500
radius=500

# Creating Dataframe of Postal Codes and Neighborhoods

In [3]:
#extract html of wikipedia page with toronto Postal Code Data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")
tables = soup.find_all('table') 

#select table of interest. Based on wikipedia page observation, this will be the first table
table_index=0

In [4]:
#create df with columns as specified in the assignment
t_hoods = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])


#iterate thorugh each row
for row in tables[table_index].tbody.find_all("tr"):
    
    
    #find all cells in the currently iterated row
    col = row.find_all("td")
    
    
    #if the cell is not blank then for every cell, append the contents to the dataframe in the 'PostalCode' column
    if (col != []):
        for i in range(0,len(col)):
            postalcode=col[i].text.strip()
            borough = ''
            neighborhood = ''
            t_hoods = t_hoods.append({"PostalCode":postalcode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)

            
#Populate the 'Borouhgs' column with 'PoistalCode' column values after stripping the first 3 letters
#(this is the postal code based on the data structure)          
t_hoods['Borough']=t_hoods['PostalCode'].str[3:]


#create a list from the 'Boroughs' column and iterate through the list,
#splitting the string on an opening parentheses and saving this into another list. 
#Then replace the Boroughs column with these updated list values, 
#these are the Boroughs separated from the neighborhoods and postal codes 
breakoff = list(t_hoods["Borough"]) 
bors=[]
for i in breakoff:
    bors.append(i.split('(')[0])
t_hoods['Borough']=bors


#from the unbroken 'PostalCodes' string, extract the text within parenthese. 
#based on the data structure these are the Neighborhoods 
t_hoods['Neighborhood']=t_hoods['PostalCode'].str.extract('.*\((.*)\).*')


#create a new df from the current, excluding any rows where the 'Borough' entry is 'Not assigned'
t_hoods=t_hoods[~t_hoods.Borough.str.contains('Not assigned')]


#create a list from the 'Neighborhood' column and iterate through the list,
#replace ' /' with ',' for formatting as directed by the assignment 
#Then replace the Neighborhood column with these updated list values
hood=[]
hoods=list(t_hoods['Neighborhood'])
for i in hoods:
    hood.append(i.replace(' /',','))
t_hoods['Neighborhood']=hood

#Strip all text except the first 3 characters from the 'PostalCodes' column
#this leaves the column as the postal codes based on the datastructure
t_hoods['PostalCode']=t_hoods['PostalCode'].str[0:3]

#reset index
t_hoods=t_hoods.reset_index(drop=True)

#display the size of the df and head of 12 rows as requested by the assignment
print('DataFrame Size is',t_hoods.shape)
t_hoods.head(12)

DataFrame Size is (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Adding Geolocation Data to Dataframe

In [5]:
# import geocoder


# lat_lng_coords = None

# for i in postal_codes:
#     while(lat_lng_coords is None):
#       g = geocoder.google('M9B, Toronto, Ontario')
#       lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


#THE ABOVE SECTION WOULD NOT WORK WITHOUT A LOOP, THE LOOP ATTEMPT IS NOT SHOWN HERE BUT FAILED ALSO
#THIS ASSIGNMENT WILL PROCEED WITH THE PROVIDED .csv FILE FOR GEOLOCATION DATA

#Import io library and get geolocation data from the provides csv into a pandas dataframe

data = requests.get('http://cocl.us/Geospatial_data',allow_redirects=True).text
data_file = io.StringIO(data)
geodata = pd.read_csv(data_file)
geodata = geodata.reset_index().set_index('Postal Code')

#Create ampty lists to add latitude and longitude values, iterate through existing dataframe and match postal codes,
#adding latitude and longitude data in order to lists, after all postal codes are looped through 
#add lists to dataframe and concatenate dataframe to dataframe created in previous cell

lat=[]
long=[]

for i in t_hoods['PostalCode']:
    lat.append(geodata.loc[i,'Latitude'])
    long.append(geodata.loc[i,'Longitude'])

d={'Latitude':lat,'Longitude':long}
df=pd.DataFrame(d)

t_hoods=pd.concat([t_hoods,df],axis=1)

#Split rows with multiple neighborhoods retaining the rest of row data
t_hoods=t_hoods.drop('Neighborhood', axis=1).join(t_hoods['Neighborhood'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('Neighborhood'))

#Wikipedia source had several outlier datapoints, replace these points with the intended text
#Ideally this could be done with a text search loop and split, however with the limited number of occurrences this was executed manually
t_hoods['Borough']=t_hoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(t_hoods['Borough'].unique())

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East York/East Toronto'
 'Central Toronto' 'Mississauga' 'Downtown Toronto Stn A'
 'Etobicoke Northwest' 'East Toronto Business']


# Visualizing Neighborhoods

In [6]:
#Install additional libraries for mapping
!pip install matplotlib
!pip install sklearn
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
!pip install geopy
from geopy.geocoders import Nominatim

In [7]:
#Get lat and long of Toronto for backdrop map
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# # add markers to map
# for lat, lng, borough, neighborhood in zip(t_hoods['Latitude'], t_hoods['Longitude'], t_hoods['Borough'], t_hoods['Neighborhood']):
#     label = '{}, {}'.format(neighborhood, borough)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_toronto)  
    
# map_toronto

In [8]:
#Set up number of clusters and drop text columns to cluster locational data
kclusters = 5

t_hoods_clust = t_hoods.drop(['Neighborhood','PostalCode','Borough'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_hoods_clust)

#add cluster labels to df 
t_hoods.insert(5, 'Cluster Labels', kmeans.labels_)

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map colored by cluster label
markers_colors = []
for lat, lon, poi, cluster in zip(t_hoods['Latitude'], t_hoods['Longitude'], t_hoods['Neighborhood'], t_hoods['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters